In [1]:
import json
import os
import pandas as pd
import re

C:\Users\Thorn\AppData\Local\Temp\ipykernel_9220\819515313.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import spacy
from spacy import displacy
from spacy.tokens import Span

D:\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import TFDebertaForTokenClassification, DebertaTokenizer

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

In [ ]:
import torch

# Load Data

In [ ]:
path_train = './in/train.json'
path_test = './in/test.json'

In [ ]:
train_json = json.load(open(path_train))
df_train = pd.json_normalize(train_json)

test_json = json.load(open(path_test))
df_test = pd.json_normalize(test_json)

In [ ]:
train_json[0].keys()

In [ ]:
df_train.head()

In [ ]:
text_test = """Meet Jane Doe, a brilliant student at XYZ University. She can be reached at jane.doe@email.com or through her phone number +1234567890. Jane resides at 123 Main Street, Cityville. Her student ID is 987654 and her personal website is www.janedoe.com. Connect with her on social media using the username @janedoe.

Meanwhile, John Smith, another outstanding student, can be contacted at john.smith@email.com or at +9876543210. John lives at 456 Oak Avenue, Townsville. His student ID is 123456, and you can visit his personal blog at www.johnsmithblog.com. Follow him on Twitter with the handle @johnsmith123.

For any inquiries about the university's programs, you can contact the administration office at admin@xyzuniversity.edu or call +5551234567. The office is located at 789 University Boulevard.

Visit our official website at www.xyzuniversity.edu for more information on courses and admission procedures.

"""


In [ ]:
text_test

# Model

## Run (lakshyakh93/deberta_finetuned_pii)

In [ ]:
full_text = df_train.loc[0].full_text
text = full_text
model_name = "lakshyakh93/deberta_finetuned_pii"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

inputs = tokenizer(text, add_special_tokens=True, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_token_class_ids = logits.argmax(-1)

# Note that tokens are classified rather then input words which means that
# there might be more predicted token classes than words.
# Multiple token classes might account for the same word
predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]

labels = predicted_token_class_ids
loss = model(**inputs, labels=labels).loss

predicted_tokens_classes

## Pipeline

In [ ]:
from transformers import pipeline
gen = pipeline("token-classification", "lakshyakh93/deberta_finetuned_pii", device=-1)

# temp_text = "My name is John and I live in California."
temp_text = text_test
output = gen(temp_text, aggregation_strategy="first")

In [105]:
text_test

"Meet Jane Doe, a brilliant student at XYZ University. She can be reached at jane.doe@email.com or through her phone number +1234567890. Jane resides at 123 Main Street, Cityville. Her student ID is 987654 and her personal website is www.janedoe.com. Connect with her on social media using the username @janedoe.\n\nMeanwhile, John Smith, another outstanding student, can be contacted at john.smith@email.com or at +9876543210. John lives at 456 Oak Avenue, Townsville. His student ID is 123456, and you can visit his personal blog at www.johnsmithblog.com. Follow him on Twitter with the handle @johnsmith123.\n\nFor any inquiries about the university's programs, you can contact the administration office at admin@xyzuniversity.edu or call +5551234567. The office is located at 789 University Boulevard.\n\nVisit our official website at www.xyzuniversity.edu for more information on courses and admission procedures.\n\n"

In [108]:
output[16:]

[{'entity_group': 'ZIPCODE',
  'score': 0.59201986,
  'word': ' 123456,',
  'start': 484,
  'end': 492},
 {'entity_group': 'URL',
  'score': 0.72377163,
  'word': ' www.johnsmithblog.com.',
  'start': 531,
  'end': 554},
 {'entity_group': 'EMAIL',
  'score': 0.9881231,
  'word': ' admin@xyzuniversity.edu',
  'start': 704,
  'end': 728},
 {'entity_group': 'PHONE_NUMBER',
  'score': 0.9487186,
  'word': ' +5551234567.',
  'start': 736,
  'end': 749},
 {'entity_group': 'STREETADDRESS',
  'score': 0.9893532,
  'word': ' 789 University Boulevard.\n\nVisit',
  'start': 774,
  'end': 807},
 {'entity_group': 'URL',
  'score': 0.9544076,
  'word': ' www.xyzuniversity.edu',
  'start': 831,
  'end': 853}]

In [24]:
type(tokens.word_ids())

list

In [27]:
tokens.tokens()

['[CLS]', 'your', 'original', 'sentence', 'here', '.', '[SEP]']

In [25]:
tokens.word_ids()

[None, 0, 1, 2, 3, 4, None]

In [40]:
inputs.tokens()

['[CLS]',
 'ĠDesign',
 'ĠThinking',
 'Ġfor',
 'Ġinnovation',
 'Ġreflex',
 'ion',
 '-',
 'Av',
 'ril',
 'Ġ2021',
 '-',
 'N',
 'ath',
 'al',
 'ie',
 'ĠSy',
 'lla',
 'Ċ',
 'Ċ',
 'Chall',
 'enge',
 'Ġ&',
 'Ġselection',
 'Ċ',
 'Ċ',
 'The',
 'Ġtool',
 'ĠI',
 'Ġuse',
 'Ġto',
 'Ġhelp',
 'Ġall',
 'Ġstakeholders',
 'Ġfinding',
 'Ġtheir',
 'Ġway',
 'Ġthrough',
 'Ġthe',
 'Ġcomplexity',
 'Ġof',
 'Ġa',
 'Ġproject',
 'Ġis',
 'Ġthe',
 'Ġ',
 'Ġmind',
 'Ġmap',
 '.',
 'Ċ',
 'Ċ',
 'What',
 'Ġexactly',
 'Ġis',
 'Ġa',
 'Ġmind',
 'Ġmap',
 '?',
 'ĠAccording',
 'Ġto',
 'Ġthe',
 'Ġdefinition',
 'Ġof',
 'ĠBu',
 'zan',
 'ĠT',
 '.',
 'Ġand',
 'ĠBu',
 'zan',
 'ĠB',
 '.',
 'Ġ(',
 '1999',
 ',',
 'ĠD',
 'ess',
 'ine',
 '-',
 'mo',
 'i',
 'Ġ',
 'Ġl',
 "'",
 'intelligence',
 '.',
 'ĠParis',
 ':',
 'ĠLes',
 'ĠÃī',
 'd',
 'itions',
 'Ġd',
 "'",
 'Organ',
 'isation',
 '.),',
 'Ġthe',
 'Ġmind',
 'Ġmap',
 'Ġ(',
 'or',
 'Ġhe',
 'uristic',
 'Ġdiagram',
 ')',
 'Ġis',
 'Ġa',
 'Ġgraphic',
 'Ġ',
 'Ġrepresentation',
 'Ġtechnique',


In [17]:
predicted_tokens_classes

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O'

# Experiment

## Spacy tokenizer

In [ ]:
import spacy

# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Sample text to tokenize
text = "Apple is looking at buying U.K. startup for $1 billion."

# Tokenize the text using spaCy
doc = nlp(text)

# Reconstruct the original text
reconstructed_text = ''.join(token.text_with_ws for token in doc)

print(reconstructed_text)

In [ ]:
''.join([token.text + token.whitespace_  for token in doc])

# Parts

In [ ]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

def ner_pipeline(text):
    # Load pre-trained BERT model and tokenizer for token classification
    model_name = "bert-base-uncased"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForTokenClassification.from_pretrained(model_name)

    # Tokenize the text using the model's tokenizer
    inputs = tokenizer(text, return_tensors="pt", truncation=True)

    # Run inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted labels
    predictions = torch.argmax(outputs.logits, dim=2)

    # Convert predicted labels to list
    predicted_labels = predictions[0].tolist()

    # Align the predicted labels to the original words
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    words = tokenizer.convert_tokens_to_string(tokens).split()

    # Create a list of tuples (word, predicted_label)
    word_label_pairs = list(zip(words, predicted_labels))

    return word_label_pairs

# # Example usage
# text = "Named Entity Recognition with BERT is great!"
# result = ner_pipeline(text)
print(result)

In [ ]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

text = "Named Entity Recognition with BERT is great Sylvian!"
# Load pre-trained BERT model and tokenizer for token classification
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

# Tokenize the text using the model's tokenizer
inputs = tokenizer(text, return_tensors="pt", truncation=True)

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted labels
predictions = torch.argmax(outputs.logits, dim=2)

# Convert predicted labels to list
predicted_labels = predictions[0].tolist()

# Align the predicted labels to the original words
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
words = tokenizer.convert_tokens_to_string(tokens).split()

# Create a list of tuples (word, predicted_label)
word_label_pairs = list(zip(words, predicted_labels))

# # Example usage

# result = ner_pipeline(text)
print(word_label_pairs)

In [90]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

['[CLS]',
 'named',
 'entity',
 'recognition',
 'with',
 'bert',
 'is',
 'great',
 'sylvia',
 '##n',
 '!',
 '[SEP]']

In [95]:
predicted_labels

[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1]

In [94]:
len(words)

11

In [93]:
len(inputs["input_ids"][0])

12

In [91]:
len(predicted_labels)

12

In [85]:
inputs.tokens()

ValueError: tokens() is not available when using non-fast tokenizers (e.g. instance of a `XxxTokenizerFast` class).

## Token Limit (temp)

In [ ]:
full_text = df_train.loc[0].full_text
nlp = spacy.load("en_core_web_sm")
doc = nlp(full_text)
text = ''.join([token.text_with_ws for token in doc[:100]])

In [15]:
text

"Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection\n\nThe tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.\n\nWhat exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain"

## Highest non-O probability:

In [ ]:
def get_highest_non_O_labels_threshold(tokens, predicted_labels, label_probabilities, threshold=0.5):
    highest_non_O_labels = []

    for token, label_ids, probabilities in zip(tokens, predicted_labels, label_probabilities):
        labels = [model.config.id2label[label_id] for label_id in label_ids]
        non_O_labels = []
        non_O_probabilities = []

        # Iterate through predicted labels and their probabilities
        for label, probability in zip(labels, probabilities):
            # Exclude 'O' labels and labels with probabilities below the threshold
            if label != 'O' and probability >= threshold:
                non_O_labels.append(label)
                non_O_probabilities.append(probability)

        # If there are non-'O' labels above the threshold, select the one with the highest probability
        if non_O_labels:
            highest_non_O_label = non_O_labels[non_O_probabilities.index(max(non_O_probabilities))]
            highest_non_O_labels.append((token, highest_non_O_label))
        else:
            # If all labels are 'O' or below the threshold, consider it as non-entity
            highest_non_O_labels.append((token, 'O'))

    return highest_non_O_labels

# Set threshold
threshold = 0.5

## Prediction Alignment

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import spacy

# Load pre-trained BERT model and tokenizer using Auto classes
model_name = "lakshyakh93/deberta_finetuned_pii"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Load spaCy model for tokenization
nlp = spacy.load("en_core_web_sm")

# Sample text
# text = "John Smith works at OpenAI in San Francisco."

# text = df_train.loc[0].full_text
# text = text
text = text_test

doc = nlp(text)

# Step 1: Tokenize the text using spaCy for words
words = [token.text for token in doc]

# Step 2: Tokenize the text using the model's tokenizer and get word to subword mapping
# inputs = tokenizer(text, return_tensors='pt', return_offsets_mapping=False)
inputs = tokenizer(words, return_tensors='pt', return_offsets_mapping=False, is_split_into_words=True)
word_ids = inputs.word_ids()

# Step 3: Run inference
with torch.no_grad():
    outputs = model(**inputs).logits

# Step 4: Create a list of which tokens or subwords correspond to a word using the word_ids variable
word_subword_mapping = {}
for i, word_id in enumerate(word_ids):
    if word_id is not None:
        if word_id not in word_subword_mapping:
            word_subword_mapping[word_id] = []
        word_subword_mapping[word_id].append(i)

# Instantiate predictions variable
predictions = torch.argmax(outputs, dim=2)

# Step 5: Iterate through pairs of words and subwords to count the majority label
word_labels = []
for word_id, subword_indices in word_subword_mapping.items():
    subword_labels = predictions[0, subword_indices]
    majority_label = torch.mode(subword_labels).values.item()
    word_labels.append((words[word_id], model.config.id2label[majority_label]))

# Display results
# print("Original Text:", text)
# print("Word Labels:", word_labels)

In [23]:
label_probabilities = torch.softmax(outputs, dim=2)[0].cpu().numpy()

In [30]:
len(label_probabilities[4])

116

In [19]:
outputs[0][0]

tensor([ 2.9995e-01,  7.0126e-01,  6.4311e-01,  2.6335e-01,  8.8299e-01,
         4.3662e-01,  7.0182e-01,  4.0063e+00,  7.9710e-01, -2.3113e-02,
         2.4447e-01, -2.1183e-01, -3.6750e-01,  1.4272e+00, -7.0485e-01,
        -1.2629e-01, -2.1296e-01, -5.5472e-02, -3.4729e-01,  7.6680e-02,
        -5.6008e-01, -7.2453e-01, -8.7485e-01,  2.6355e-01, -4.3632e-01,
        -4.6025e-02, -3.5645e-01, -7.4701e-01, -8.6652e-01, -7.8131e-01,
        -1.2693e+00, -5.5256e-01, -4.4496e-01,  1.9675e-01, -1.2938e+00,
        -7.4463e-01, -4.8393e-01,  4.2690e-01,  5.4200e-01,  2.0513e-01,
        -3.3553e-03,  4.3093e-01, -5.1900e-01, -4.4200e-01, -8.3023e-01,
        -8.0837e-01, -1.4081e+00, -8.1950e-01, -1.1582e+00, -2.8280e-01,
         1.3150e-01,  3.8448e-02, -1.0471e+00, -5.3441e-01,  2.4152e-04,
         4.7095e-02, -1.3062e+00, -1.2903e+00,  2.4801e-01,  1.1354e+00,
        -2.8463e-01,  6.2392e-01, -5.0984e-01, -4.8388e-01, -7.3426e-01,
         2.1283e-02,  3.7291e-02,  6.6526e-02,  9.2

In [13]:
outputs

NameError: name 'outputs' is not defined

In [94]:
word_ids[:20]

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, 13, 14, 15, 16, 16]

In [93]:
list_tokens[:20]

['[CLS]',
 'ĠDesign',
 'ĠThinking',
 'Ġfor',
 'Ġinnovation',
 'Ġreflex',
 'ion',
 'Ġ-',
 'ĠAv',
 'ril',
 'Ġ2021',
 'Ġ-',
 'ĠNath',
 'al',
 'ie',
 'ĠSy',
 'lla',
 'Ġ',
 'ĊĊ',
 'ĠChallenge']

In [75]:
word_labels

[('Design', 'O'),
 ('Thinking', 'O'),
 ('for', 'O'),
 ('innovation', 'O'),
 ('reflexion', 'B-FIRSTNAME'),
 ('-', 'O'),
 ('Avril', 'I-FIRSTNAME'),
 ('2021', 'B-NUMBER'),
 ('-', 'I-NUMBER'),
 ('Nathalie', 'I-NUMBER'),
 ('Sylla', 'I-NUMBER'),
 ('\n\n', 'O'),
 ('Challenge', 'O'),
 ('&', 'O'),
 ('selection', 'O'),
 ('\n\n', 'O'),
 ('The', 'O'),
 ('tool', 'O'),
 ('I', 'O'),
 ('use', 'O'),
 ('to', 'O'),
 ('help', 'O'),
 ('all', 'O'),
 ('stakeholders', 'O'),
 ('finding', 'O'),
 ('their', 'O'),
 ('way', 'O'),
 ('through', 'O'),
 ('the', 'O'),
 ('complexity', 'O'),
 ('of', 'O'),
 ('a', 'O'),
 ('project', 'O'),
 ('is', 'O'),
 ('the', 'O'),
 (' ', 'O'),
 ('mind', 'O'),
 ('map', 'O'),
 ('.', 'O'),
 ('\n\n', 'O'),
 ('What', 'O'),
 ('exactly', 'O'),
 ('is', 'O'),
 ('a', 'O'),
 ('mind', 'O'),
 ('map', 'O'),
 ('?', 'O'),
 ('According', 'O'),
 ('to', 'O'),
 ('the', 'O'),
 ('definition', 'O'),
 ('of', 'O'),
 ('Buzan', 'O'),
 ('T.', 'O'),
 ('and', 'O'),
 ('Buzan', 'O'),
 ('B.', 'O'),
 ('(', 'O'),
 ('1999'

In [63]:
print(f'len_tokens = {len(list_tokens)} | len_pred = {len(predictions[0])}')

len_tokens = 125 | len_pred = 125


In [61]:
predictions[0][:20]

tensor([  7,   7,   7,   7,   7,   2,   2,   7,   3,   3, 107, 108, 108, 108,
        108, 108, 108,   7,   7,   7])

In [72]:
pred_labels = [model.config.id2label[int(pred)] for pred in predictions[0]]

In [74]:
token_labels = list(zip(list_tokens, pred_labels))
token_labels

[('[CLS]', 'O'),
 ('ĠDesign', 'O'),
 ('ĠThinking', 'O'),
 ('Ġfor', 'O'),
 ('Ġinnovation', 'O'),
 ('Ġreflex', 'B-FIRSTNAME'),
 ('ion', 'B-FIRSTNAME'),
 ('Ġ-', 'O'),
 ('ĠAv', 'I-FIRSTNAME'),
 ('ril', 'I-FIRSTNAME'),
 ('Ġ2021', 'B-NUMBER'),
 ('Ġ-', 'I-NUMBER'),
 ('ĠNath', 'I-NUMBER'),
 ('al', 'I-NUMBER'),
 ('ie', 'I-NUMBER'),
 ('ĠSy', 'I-NUMBER'),
 ('lla', 'I-NUMBER'),
 ('Ġ', 'O'),
 ('ĊĊ', 'O'),
 ('ĠChallenge', 'O'),
 ('Ġ&', 'O'),
 ('Ġselection', 'O'),
 ('Ġ', 'O'),
 ('ĊĊ', 'O'),
 ('ĠThe', 'O'),
 ('Ġtool', 'O'),
 ('ĠI', 'O'),
 ('Ġuse', 'O'),
 ('Ġto', 'O'),
 ('Ġhelp', 'O'),
 ('Ġall', 'O'),
 ('Ġstakeholders', 'O'),
 ('Ġfinding', 'O'),
 ('Ġtheir', 'O'),
 ('Ġway', 'O'),
 ('Ġthrough', 'O'),
 ('Ġthe', 'O'),
 ('Ġcomplexity', 'O'),
 ('Ġof', 'O'),
 ('Ġa', 'O'),
 ('Ġproject', 'O'),
 ('Ġis', 'O'),
 ('Ġthe', 'O'),
 ('Ġ', 'O'),
 ('Ġmind', 'O'),
 ('Ġmap', 'O'),
 ('Ġ.', 'O'),
 ('Ġ', 'O'),
 ('ĊĊ', 'O'),
 ('ĠWhat', 'O'),
 ('Ġexactly', 'O'),
 ('Ġis', 'O'),
 ('Ġa', 'O'),
 ('Ġmind', 'O'),
 ('Ġmap', 'O'),
 ('Ġ

### Compare

In [77]:
predictions[0][:20]

tensor([  7,   7,   7,   7,   7,   2,   2,   7,   7,   7, 107,  19,  19,  19,
         19,  19,   4,   4,   5,   7])

In [78]:
pred_labels = [model.config.id2label[int(pred)] for pred in predictions[0]]

In [79]:
token_labels = list(zip(list_tokens, pred_labels))
token_labels

[('[CLS]', 'O'),
 ('ĠDesign', 'O'),
 ('ĠThinking', 'O'),
 ('Ġfor', 'O'),
 ('Ġinnovation', 'O'),
 ('Ġreflex', 'B-FIRSTNAME'),
 ('ion', 'B-FIRSTNAME'),
 ('Ġ-', 'O'),
 ('ĠAv', 'O'),
 ('ril', 'O'),
 ('Ġ2021', 'B-NUMBER'),
 ('Ġ-', 'I-DATE'),
 ('ĠNath', 'I-DATE'),
 ('al', 'I-DATE'),
 ('ie', 'I-DATE'),
 ('ĠSy', 'I-DATE'),
 ('lla', 'B-MIDDLENAME'),
 ('Ġ', 'B-MIDDLENAME'),
 ('ĊĊ', 'B-LASTNAME'),
 ('ĠChallenge', 'O'),
 ('Ġ&', 'O'),
 ('Ġselection', 'O'),
 ('Ġ', 'O'),
 ('ĊĊ', 'O'),
 ('ĠThe', 'O'),
 ('Ġtool', 'O'),
 ('ĠI', 'O'),
 ('Ġuse', 'O'),
 ('Ġto', 'O'),
 ('Ġhelp', 'O'),
 ('Ġall', 'O'),
 ('Ġstakeholders', 'O'),
 ('Ġfinding', 'O'),
 ('Ġtheir', 'O'),
 ('Ġway', 'O'),
 ('Ġthrough', 'O'),
 ('Ġthe', 'O'),
 ('Ġcomplexity', 'O'),
 ('Ġof', 'O'),
 ('Ġa', 'O'),
 ('Ġproject', 'O'),
 ('Ġis', 'O'),
 ('Ġthe', 'O'),
 ('Ġ', 'O'),
 ('Ġmind', 'O'),
 ('Ġmap', 'O'),
 ('Ġ.', 'O'),
 ('Ġ', 'O'),
 ('ĊĊ', 'O'),
 ('ĠWhat', 'O'),
 ('Ġexactly', 'O'),
 ('Ġis', 'O'),
 ('Ġa', 'O'),
 ('Ġmind', 'O'),
 ('Ġmap', 'O'),
 ('Ġ?', 'O

In [56]:
word_labels

[('Design', 'O'),
 ('Thinking', 'O'),
 ('for', 'O'),
 ('innovation', 'O'),
 ('reflexion', 'B-FIRSTNAME'),
 ('-', 'O'),
 ('Avril', 'I-FIRSTNAME'),
 ('2021', 'B-NUMBER'),
 ('-', 'I-NUMBER'),
 ('Nathalie', 'I-NUMBER'),
 ('Sylla', 'I-NUMBER'),
 ('\n\n', 'O'),
 ('Challenge', 'O'),
 ('&', 'O'),
 ('selection', 'O'),
 ('\n\n', 'O'),
 ('The', 'O'),
 ('tool', 'O'),
 ('I', 'O'),
 ('use', 'O'),
 ('to', 'O'),
 ('help', 'O'),
 ('all', 'O'),
 ('stakeholders', 'O'),
 ('finding', 'O'),
 ('their', 'O'),
 ('way', 'O'),
 ('through', 'O'),
 ('the', 'O'),
 ('complexity', 'O'),
 ('of', 'O'),
 ('a', 'O'),
 ('project', 'O'),
 ('is', 'O'),
 ('the', 'O'),
 (' ', 'O'),
 ('mind', 'O'),
 ('map', 'O'),
 ('.', 'O'),
 ('\n\n', 'O'),
 ('What', 'O'),
 ('exactly', 'O'),
 ('is', 'O'),
 ('a', 'O'),
 ('mind', 'O'),
 ('map', 'O'),
 ('?', 'O'),
 ('According', 'O'),
 ('to', 'O'),
 ('the', 'O'),
 ('definition', 'O'),
 ('of', 'O'),
 ('Buzan', 'O'),
 ('T.', 'O'),
 ('and', 'O'),
 ('Buzan', 'O'),
 ('B.', 'O'),
 ('(', 'O'),
 ('1999'

In [55]:
words[:20]

['Design',
 'Thinking',
 'for',
 'innovation',
 'reflexion',
 '-',
 'Avril',
 '2021',
 '-',
 'Nathalie',
 'Sylla',
 '\n\n',
 'Challenge',
 '&',
 'selection',
 '\n\n',
 'The',
 'tool',
 'I',
 'use']

In [53]:
list_tokens[:20]

['[CLS]',
 'ĠDesign',
 'ĠThinking',
 'Ġfor',
 'Ġinnovation',
 'Ġreflex',
 'ion',
 'Ġ-',
 'ĠAv',
 'ril',
 'Ġ2021',
 'Ġ-',
 'ĠNath',
 'al',
 'ie',
 'ĠSy',
 'lla',
 'Ġ',
 'ĊĊ',
 'ĠChallenge']

In [43]:
token_v[4]

'reflexion'

In [34]:
word_subword_mapping

{0: [1],
 1: [2],
 2: [3],
 3: [4],
 4: [5, 6],
 5: [7],
 6: [8, 9],
 7: [10],
 8: [11],
 9: [12, 13, 14],
 10: [15, 16],
 11: [17, 18],
 12: [19],
 13: [20],
 14: [21],
 15: [22, 23],
 16: [24],
 17: [25],
 18: [26],
 19: [27],
 20: [28],
 21: [29],
 22: [30],
 23: [31],
 24: [32],
 25: [33],
 26: [34],
 27: [35],
 28: [36],
 29: [37],
 30: [38],
 31: [39],
 32: [40],
 33: [41],
 34: [42],
 35: [43],
 36: [44],
 37: [45],
 38: [46],
 39: [47, 48],
 40: [49],
 41: [50],
 42: [51],
 43: [52],
 44: [53],
 45: [54],
 46: [55],
 47: [56],
 48: [57],
 49: [58],
 50: [59],
 51: [60],
 52: [61, 62],
 53: [63, 64],
 54: [65],
 55: [66, 67],
 56: [68, 69],
 57: [70],
 58: [71],
 59: [72],
 60: [73, 74, 75],
 61: [76],
 62: [77, 78],
 63: [79],
 64: [80, 81, 82],
 65: [83],
 66: [84],
 67: [85],
 68: [86],
 69: [87, 88, 89],
 70: [90, 91, 92, 93],
 71: [94],
 72: [95],
 73: [96],
 74: [97],
 75: [98],
 76: [99],
 77: [100],
 78: [101],
 79: [102, 103],
 80: [104],
 81: [105],
 82: [106],
 83: [1

In [14]:
token_v, pred_v = zip(*word_labels)

In [64]:
list_words = [token.text_with_ws for token in doc[:100]]
list_tokens = inputs.tokens()

In [37]:
# Convert tokens back to words
text_recon = tokenizer.convert_tokens_to_string(list_tokens)

# Split the sentence into a list of words
list_words_2 = text_recon.split()
list_words_2_adj = list_words_2[1:]

In [36]:
list_words_2

['[CLS]',
 'Design',
 'Thinking',
 'for',
 'innovation',
 'reflexion-Avril',
 '2021-Nathalie',
 'Sylla',
 'Challenge',
 '&',
 'selection',
 'The',
 'tool',
 'I',
 'use',
 'to',
 'help',
 'all',
 'stakeholders',
 'finding',
 'their',
 'way',
 'through',
 'the',
 'complexity',
 'of',
 'a',
 'project',
 'is',
 'the',
 'mind',
 'map.',
 'What',
 'exactly',
 'is',
 'a',
 'mind',
 'map?',
 'According',
 'to',
 'the',
 'definition',
 'of',
 'Buzan',
 'T.',
 'and',
 'Buzan',
 'B.',
 '(1999,',
 'Dessine-moi',
 "l'intelligence.",
 'Paris:',
 'Les',
 'Éditions',
 "d'Organisation.),",
 'the',
 'mind',
 'map',
 '(or',
 'heuristic',
 'diagram)',
 'is',
 'a',
 'graphic',
 'representation',
 'technique',
 'that',
 'follows',
 'the',
 'natural',
 'functioning',
 'of',
 'the',
 'mind',
 'and',
 'allows',
 'the',
 'brain[SEP]']

In [27]:
for i in range(len(word_subword_mapping)):
    list_j = word_subword_mapping[i]
    subwords = [list_tokens[j] for j in list_j]
    print(f'{i} : {list_words[i]} ===> {subwords}')

0 : Design  ===> ['ĠDesign']
1 : Thinking  ===> ['ĠThinking']
2 : for  ===> ['Ġfor']
3 : innovation  ===> ['Ġinnovation']
4 : reflexion ===> ['Ġreflex', 'ion']
5 : - ===> ['Ġ-']
6 : Avril  ===> ['ĠAv', 'ril']
7 : 2021 ===> ['Ġ2021']
8 : - ===> ['Ġ-']
9 : Nathalie  ===> ['ĠNath', 'al', 'ie']
10 : Sylla ===> ['ĠSy', 'lla']
11 : 

 ===> ['Ġ', 'ĊĊ']
12 : Challenge  ===> ['ĠChallenge']
13 : &  ===> ['Ġ&']
14 : selection ===> ['Ġselection']
15 : 

 ===> ['Ġ', 'ĊĊ']
16 : The  ===> ['ĠThe']
17 : tool  ===> ['Ġtool']
18 : I  ===> ['ĠI']
19 : use  ===> ['Ġuse']
20 : to  ===> ['Ġto']
21 : help  ===> ['Ġhelp']
22 : all  ===> ['Ġall']
23 : stakeholders  ===> ['Ġstakeholders']
24 : finding  ===> ['Ġfinding']
25 : their  ===> ['Ġtheir']
26 : way  ===> ['Ġway']
27 : through  ===> ['Ġthrough']
28 : the  ===> ['Ġthe']
29 : complexity  ===> ['Ġcomplexity']
30 : of  ===> ['Ġof']
31 : a  ===> ['Ġa']
32 : project  ===> ['Ġproject']
33 : is  ===> ['Ġis']
34 : the  ===> ['Ġthe']
35 :   ===> ['Ġ']
36 : mind  =

In [26]:
for i in range(len(word_subword_mapping)):
    list_j = word_subword_mapping[i]
    subwords = [list_tokens[j] for j in list_j]
    print(f'{i} : {list_words_2_adj[i]} ===> {subwords}')

NameError: name 'list_words_2_adj' is not defined

In [ ]:
list_words_2

In [13]:
word_subword_mapping

{0: [1],
 1: [2],
 2: [3],
 3: [4],
 4: [5, 6],
 5: [7],
 6: [8, 9],
 7: [10],
 8: [11],
 9: [12, 13, 14, 15],
 10: [16, 17],
 11: [18],
 12: [19],
 13: [20, 21],
 14: [22],
 15: [23],
 16: [24],
 17: [25],
 18: [26],
 19: [27],
 20: [28],
 21: [29],
 22: [30],
 23: [31],
 24: [32],
 25: [33],
 26: [34],
 27: [35],
 28: [36],
 29: [37],
 30: [38],
 31: [39],
 32: [40],
 33: [41],
 34: [42],
 35: [43],
 36: [44],
 37: [45],
 38: [46],
 39: [47],
 40: [48],
 41: [49],
 42: [50],
 43: [51],
 44: [52],
 45: [53],
 46: [54],
 47: [55],
 48: [56],
 49: [57],
 50: [58],
 51: [59],
 52: [60],
 53: [61],
 54: [62],
 55: [63, 64],
 56: [65],
 57: [66],
 58: [67],
 59: [68, 69],
 60: [70],
 61: [71],
 62: [72],
 63: [73],
 64: [74],
 65: [75, 76, 77],
 66: [78],
 67: [79, 80],
 68: [81],
 69: [82],
 70: [83],
 71: [84],
 72: [85],
 73: [86],
 74: [87],
 75: [88],
 76: [89, 90, 91],
 77: [92],
 78: [93],
 79: [94, 95],
 80: [96],
 81: [97],
 82: [98],
 83: [99],
 84: [100],
 85: [101],
 86: [102, 

### To Do: Check word-subword mappings -> indices to text

In [96]:
word_labels

[('Design', 'O'),
 ('Thinking', 'O'),
 ('for', 'O'),
 ('innovation', 'O'),
 ('reflexion', 'O'),
 ('-', 'O'),
 ('Avril', 'O'),
 ('2021', 'B-DATE'),
 ('-', 'I-DATE'),
 ('Nathalie', 'I-DATE'),
 ('Sylla', 'B-MIDDLENAME'),
 ('\n\n', 'B-LASTNAME'),
 ('Challenge', 'O'),
 ('&', 'O'),
 ('selection', 'O'),
 ('\n\n', 'O'),
 ('The', 'O'),
 ('tool', 'O'),
 ('I', 'O'),
 ('use', 'O'),
 ('to', 'O'),
 ('help', 'O'),
 ('all', 'O'),
 ('stakeholders', 'O'),
 ('finding', 'O'),
 ('their', 'O'),
 ('way', 'O'),
 ('through', 'O'),
 ('the', 'O'),
 ('complexity', 'O'),
 ('of', 'O'),
 ('a', 'O'),
 ('project', 'O'),
 ('is', 'O'),
 ('the', 'O'),
 (' ', 'O'),
 ('mind', 'O'),
 ('map', 'O'),
 ('.', 'O'),
 ('\n\n', 'O'),
 ('What', 'O'),
 ('exactly', 'O'),
 ('is', 'O'),
 ('a', 'O'),
 ('mind', 'O'),
 ('map', 'O'),
 ('?', 'O'),
 ('According', 'O'),
 ('to', 'O'),
 ('the', 'O'),
 ('definition', 'O'),
 ('of', 'O'),
 ('Buzan', 'O'),
 ('T.', 'O'),
 ('and', 'O'),
 ('Buzan', 'B-MIDDLENAME'),
 ('B.', 'O'),
 ('(', 'O'),
 ('1999', 

## Reconstruct (Visualize)

In [ ]:
def visualize_span(record):
    
    global span_infos
    
    start_pos = -1
    span_infos = []
    for label_index, label in enumerate(record['labels']):
        if re.match('^B-', label):
            start_pos = label_index
            label_group = label.split('-')[1]

        if re.match('O', label) and start_pos != -1:
            end_pos = label_index
            span_dict = {'start_pos':start_pos, 'end_pos':end_pos, 'label_group':label_group}
            span_infos.append(span_dict)
            start_pos = -1

    doc_spans = []
    nlp = spacy.blank('en')
    doc = nlp(record['full_text'])
    for span_info in span_infos:
        _span = Span(doc, span_info['start_pos'], span_info['end_pos'], span_info['label_group'])
        doc_spans.append(_span)

    doc.spans['sc'] = doc_spans

    displacy.render(doc, style = 'span')

In [28]:
def visualize_span(full_text, tokens, labels):
    
    global span_infos
    
    start_pos = -1
    span_infos = []
    for label_index, label in enumerate(labels):
        if re.match('^B-', label):
            start_pos = label_index
            label_group = label.split('-')[1]

        if re.match('O', label) and start_pos != -1:
            end_pos = label_index
            span_dict = {'start_pos':start_pos, 'end_pos':end_pos, 'label_group':label_group}
            span_infos.append(span_dict)
            start_pos = -1

    doc_spans = []
    nlp = spacy.blank('en')
    doc = nlp(full_text)
    for span_info in span_infos:
        _span = Span(doc, span_info['start_pos'], span_info['end_pos'], span_info['label_group'])
        doc_spans.append(_span)

    doc.spans['sc'] = doc_spans

    displacy.render(doc, style = 'span')

In [ ]:
list(zip(*word_labels))

In [112]:
text

"Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection\n\nThe tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.\n\nWhat exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain"

In [116]:
token_v[:15]

('Design',
 'Thinking',
 'for',
 'innovation',
 'reflexion',
 '-',
 'Avril',
 '2021',
 '-',
 'Nathalie',
 'Sylla',
 '\n\n',
 'Challenge',
 '&',
 'selection')

In [23]:
pred_v[:15]

('O',
 'O',
 'O',
 'O',
 'B-FIRSTNAME',
 'O',
 'I-FIRSTNAME',
 'B-NUMBER',
 'I-NUMBER',
 'I-NUMBER',
 'I-NUMBER',
 'O',
 'O',
 'O',
 'O')

In [118]:
word_subword_mapping

{0: [1],
 1: [2],
 2: [3],
 3: [4],
 4: [5, 6],
 5: [7],
 6: [8, 9],
 7: [10],
 8: [11],
 9: [12, 13, 14, 15],
 10: [16, 17],
 11: [18],
 12: [19],
 13: [20, 21],
 14: [22],
 15: [23],
 16: [24],
 17: [25],
 18: [26],
 19: [27],
 20: [28],
 21: [29],
 22: [30],
 23: [31],
 24: [32],
 25: [33],
 26: [34],
 27: [35],
 28: [36],
 29: [37],
 30: [38],
 31: [39],
 32: [40],
 33: [41],
 34: [42],
 35: [43],
 36: [44],
 37: [45],
 38: [46],
 39: [47],
 40: [48],
 41: [49],
 42: [50],
 43: [51],
 44: [52],
 45: [53],
 46: [54],
 47: [55],
 48: [56],
 49: [57],
 50: [58],
 51: [59],
 52: [60],
 53: [61],
 54: [62],
 55: [63, 64],
 56: [65],
 57: [66],
 58: [67],
 59: [68, 69],
 60: [70],
 61: [71],
 62: [72],
 63: [73],
 64: [74],
 65: [75, 76, 77],
 66: [78],
 67: [79, 80],
 68: [81],
 69: [82],
 70: [83],
 71: [84],
 72: [85],
 73: [86],
 74: [87],
 75: [88],
 76: [89, 90, 91],
 77: [92],
 78: [93],
 79: [94, 95],
 80: [96],
 81: [97],
 82: [98],
 83: [99],
 84: [100],
 85: [101],
 86: [102, 

In [99]:
word_labels

[('Meet', 'O'),
 ('Jane', 'B-FIRSTNAME'),
 ('Doe', 'O'),
 (',', 'O'),
 ('a', 'O'),
 ('brilliant', 'O'),
 ('student', 'O'),
 ('at', 'O'),
 ('XYZ', 'O'),
 ('University', 'O'),
 ('.', 'O'),
 ('She', 'O'),
 ('can', 'O'),
 ('be', 'O'),
 ('reached', 'O'),
 ('at', 'O'),
 ('jane.doe@email.com', 'I-EMAIL'),
 ('or', 'O'),
 ('through', 'O'),
 ('her', 'O'),
 ('phone', 'O'),
 ('number', 'O'),
 ('+1234567890', 'I-PHONE_NUMBER'),
 ('.', 'O'),
 ('Jane', 'B-FIRSTNAME'),
 ('resides', 'O'),
 ('at', 'O'),
 ('123', 'B-STREETADDRESS'),
 ('Main', 'I-STREETADDRESS'),
 ('Street', 'I-STREETADDRESS'),
 (',', 'O'),
 ('Cityville', 'B-CITY'),
 ('.', 'O'),
 ('Her', 'O'),
 ('student', 'O'),
 ('ID', 'O'),
 ('is', 'O'),
 ('987654', 'B-ZIPCODE'),
 ('and', 'O'),
 ('her', 'O'),
 ('personal', 'O'),
 ('website', 'O'),
 ('is', 'O'),
 ('www.janedoe.com', 'I-URL'),
 ('.', 'O'),
 ('Connect', 'O'),
 ('with', 'O'),
 ('her', 'O'),
 ('on', 'O'),
 ('social', 'O'),
 ('media', 'O'),
 ('using', 'O'),
 ('the', 'O'),
 ('username', 'O'),


In [98]:
span_infos

[{'start_pos': 1, 'end_pos': 2, 'label_group': 'FIRSTNAME'},
 {'start_pos': 24, 'end_pos': 25, 'label_group': 'FIRSTNAME'},
 {'start_pos': 27, 'end_pos': 30, 'label_group': 'STREETADDRESS'},
 {'start_pos': 31, 'end_pos': 32, 'label_group': 'CITY'},
 {'start_pos': 37, 'end_pos': 38, 'label_group': 'ZIPCODE'},
 {'start_pos': 54, 'end_pos': 55, 'label_group': 'MIDDLENAME'},
 {'start_pos': 60, 'end_pos': 61, 'label_group': 'LASTNAME'},
 {'start_pos': 78, 'end_pos': 81, 'label_group': 'STREETADDRESS'},
 {'start_pos': 82, 'end_pos': 83, 'label_group': 'CITY'},
 {'start_pos': 88, 'end_pos': 89, 'label_group': 'ZIPCODE'},
 {'start_pos': 107, 'end_pos': 108, 'label_group': 'USERNAME'},
 {'start_pos': 136, 'end_pos': 139, 'label_group': 'STREETADDRESS'}]

### Result

In [95]:
token_v, pred_v = zip(*word_labels)

visualize_span(text, token_v, pred_v)

In [102]:
spans

[]

In [36]:
import spacy
from spacy.tokens import Doc, Span
from spacy import displacy

# Sample data
full_string = "Apple is headquartered in Cupertino, California."
tokens = ['Apple', 'is', 'headquartered', 'in', 'Cupertino', ',', 'California', '.']
predicted_labels = ['ORG', 'O', 'O', 'O', 'LOC', 'O', 'LOC', 'O']
actual_labels = ['ORG', 'O', 'O', 'O', 'LOC', 'O', 'LOC', 'O']

# Load a spaCy model
nlp = spacy.blank("en")

# Create a spaCy Doc object
doc = Doc(nlp.vocab, words=tokens)

# Predicted entities
predicted_entities = [(predicted_labels[i], i, i+1) for i in range(len(tokens)) if predicted_labels[i] != 'O']
for label, start, end in predicted_entities:
    ent = Span(doc, start, end, label=label)
    doc.ents = list(doc.ents) + [ent]

# Actual entities
actual_entities = [(actual_labels[i], i, i+1) for i in range(len(tokens)) if actual_labels[i] != 'O']
for label, start, end in actual_entities:
    ent = Span(doc, start, end, label=label)
    doc.ents = list(doc.ents) + [ent]

# Visualize using displacy
displacy.render(doc, style='ent', page=False)

ValueError: [E1010] Unable to set entity information for token 0 which is included in more than one span in entities, blocked, missing or outside.

In [34]:
import spacy
from spacy import displacy

# Sample data
full_string = "Apple is headquartered in Cupertino, California."
tokens = ['Apple', 'is', 'headquartered', 'in', 'Cupertino', ',', 'California', '.']
labels = ['ORG', 'O', 'O', 'O', 'LOC', 'O', 'LOC', 'O']

# Combine tokens and labels into spaCy format
entities = [{'start': 0, 'end': 5, 'label': 'ORG'}, {'start': 24, 'end': 34, 'label': 'LOC'}]

# Convert tokens to a list of dictionaries
words = [{'text': token, 'tag': ''} for token in tokens]

# Create a list of tuples (text, label) for named entities
ents = [(full_string[entity['start']:entity['end']], entity['label']) for entity in entities]

# Visualize using displacy
doc = {'words': words, 'ents': ents, 'title': None}
displacy.render(doc, style='ent')


AttributeError: 'dict' object has no attribute 'ents'

# Reconstruct

In [27]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Replace 'bert-base-uncased' with the name of the model you are using
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Original string
original_string = "Your original sentence here."

# Tokenize the original string
tokens = tokenizer(original_string, return_tensors="pt")

# Forward pass to get predictions (you can replace this with your actual use case)
# outputs = model(**tokens)

with torch.no_grad():
    logits = model(**tokens).logits
    
predicted_token_class_ids = logits.argmax(-1)

# Assuming you have word_ids, replace this with your actual word_ids
word_ids = tokens.word_ids()

# [0, 0, 1, 1, 1, 2, 2, 3, 4, 4, 5]
tokens.word_ids()

# Get the original words using word_ids
original_words = [tokenizer.convert_ids_to_tokens(tokens['input_ids'][0][i].item()) for i in word_ids if i is not None]

special_token_ids = tokenizer.convert_tokens_to_ids(tokenizer.special_tokens_map.values())
filtered_words = [word for word in original_words if tokenizer.convert_tokens_to_ids([word])[0] not in special_token_ids]

# Reconstruct the original string
reconstructed_string = tokenizer.convert_tokens_to_string(filtered_words)

print("Original String:", original_string)
print("Reconstructed String:", reconstructed_string)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Original String: Your original sentence here.
Reconstructed String: your original sentence here


In [26]:
tokenizer.convert_tokens_to_string(filtered_words)

'youroriginalsentencehere'

In [25]:
filtered_words

['your', 'original', 'sentence', 'here']

In [70]:
filtered_word_ids

[None, 1, 2, 3, 4, None]

In [74]:
word_ids

[None, 0, 1, 2, 3, 4, None]

In [75]:
valid_indices

[2, 3, 4, 5]

In [13]:
# Filter out special tokens and None values
valid_indices = [i for i, word_id in enumerate(word_ids) if word_id is not None and word_id not in tokenizer.all_special_ids]

# Get the original words and predicted labels using valid_indices
# original_words = [tokenizer.convert_ids_to_tokens(word_ids[i].item()) for i in valid_indices
original_words = [tokenizer.convert_ids_to_tokens(tokens['input_ids'][0][i].item()) for i in valid_indices]

filtered_predicted_labels = [predicted_labels[i] for i in valid_indices]

# Create a dictionary to map each word to its predicted label
word_label_mapping = dict(zip(original_words, filtered_predicted_labels))

# Reconstruct the original string without special tokens
reconstructed_string = tokenizer.convert_tokens_to_string(original_words)

# Align predicted labels with the reconstructed string
aligned_labels = [word_label_mapping[word] for word in original_words]

print("Original String:", original_string)
print("Reconstructed String:", reconstructed_string)
print("Aligned Labels:", aligned_labels)

NameError: name 'predicted_labels' is not defined

In [17]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import spacy

# Load pre-trained BERT model and tokenizer using Auto classes
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Load spaCy model for tokenization
nlp = spacy.load("en_core_web_sm")

# Sample text
text = "John Smith works at OpenAI in San Francisco."

# Step 1: Tokenize the text using spaCy for words
words = [token.text for token in nlp(text)]

# Step 2: Tokenize the text using the model's tokenizer and get word to subword mapping
inputs = tokenizer(text, return_tensors='pt', return_offsets_mapping=False)
word_ids = inputs.word_ids()

# Step 3: Run inference
outputs = model(**inputs).logits

# Step 4: Create a list of which tokens or subwords correspond to a word using the word_ids variable
word_subword_mapping = {}
for i, word_id in enumerate(word_ids):
    if word_id is not None:
        if word_id not in word_subword_mapping:
            word_subword_mapping[word_id] = []
        word_subword_mapping[word_id].append(i)

# Instantiate predictions variable
predictions = torch.argmax(outputs, dim=2)

# Step 5: Iterate through pairs of words and subwords to count the majority label
word_labels = []
for word_id, subword_indices in word_subword_mapping.items():
    subword_labels = predictions[0, subword_indices]
    majority_label = torch.mode(subword_labels).values.item()
    word_labels.append((words[word_id], model.config.id2label[majority_label]))

# Display results
print("Original Text:", text)
print("Word Labels:", word_labels)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Original Text: John Smith works at OpenAI in San Francisco.
Word Labels: [('John', 'LABEL_1'), ('Smith', 'LABEL_1'), ('works', 'LABEL_1'), ('at', 'LABEL_1'), ('OpenAI', 'LABEL_1'), ('in', 'LABEL_1'), ('San', 'LABEL_1'), ('Francisco', 'LABEL_0'), ('.', 'LABEL_0')]


In [78]:
valid_indices

[2, 3, 4, 5]

In [14]:
# Get the predicted labels from the output of the model
predicted_labels = torch.argmax(outputs.logits, dim=2).squeeze().tolist()

# Filter out special tokens (those with token type ID = 0)
special_token_ids = tokenizer.convert_tokens_to_ids(tokenizer.special_tokens_map.values())
filtered_word_ids = [word_id for word_id in word_ids if word_id not in special_token_ids]

# Get the original words using filtered_word_ids
original_words = [tokenizer.convert_ids_to_tokens(word_id.item()) for word_id in filtered_word_ids]

def filter_post(token):
    if token is not None and token
        return True
    
    else:
        return False

# Filter predicted labels to maintain alignment
filtered_predicted_labels = [predicted_label for predicted_label, word_id in zip(predicted_labels, word_ids) if word_id not in special_token_ids]

# Create a dictionary to map each word to its predicted label
word_label_mapping = dict(zip(original_words, filtered_predicted_labels))

# Reconstruct the original string without special tokens
reconstructed_string = tokenizer.convert_tokens_to_string(original_words)

# Align predicted labels with the reconstructed string
aligned_labels = [word_label_mapping[word] for word in original_words]

print("Original String:", original_string)
print("Reconstructed String:", reconstructed_string)
print("Aligned Labels:", aligned_labels)

SyntaxError: expected ':' (475915208.py, line 12)

In [66]:
tokens.tokens()

['[CLS]', 'your', 'original', 'sentence', 'here', '.', '[SEP]']

In [46]:
with torch.no_grad():
    logits = model(**tokens).logits
    
predicted_labels = logits.argmax(-1)

In [58]:
predicted_token_class_ids[0]

tensor([0, 1, 1, 1, 0, 0, 1])

In [55]:
torch.argmax(outputs.logits, dim=2).squeeze().tolist()

[0, 1, 1, 1, 0, 0, 1]

## Chunking

In [50]:
len(predicted_tokens_classes)

835

In [36]:
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
tokens

['[CLS]',
 '▁Design',
 '▁Thinking',
 '▁for',
 '▁innovation',
 '▁reflex',
 'ion',
 '-',
 'Av',
 'ril',
 '▁2021',
 '-',
 'N',
 'atha',
 'lie',
 '▁S',
 'ylla',
 '▁Challenge',
 '▁&',
 '▁selection',
 '▁The',
 '▁tool',
 '▁I',
 '▁use',
 '▁to',
 '▁help',
 '▁all',
 '▁stakeholders',
 '▁finding',
 '▁their',
 '▁way',
 '▁through',
 '▁the',
 '▁complexity',
 '▁of',
 '▁a',
 '▁project',
 '▁is',
 '▁the',
 '▁mind',
 '▁map',
 '.',
 '▁What',
 '▁exactly',
 '▁is',
 '▁a',
 '▁mind',
 '▁map',
 '?',
 '▁According',
 '▁to',
 '▁the',
 '▁definition',
 '▁of',
 '▁Buz',
 'an',
 '▁T',
 '.',
 '▁and',
 '▁Buz',
 'an',
 '▁B',
 '.',
 '▁(',
 '1999',
 ',',
 '▁Des',
 's',
 'ine',
 '-',
 'moi',
 '▁l',
 "'",
 'intelligence',
 '.',
 '▁Paris',
 ':',
 '▁Les',
 '▁É',
 'dition',
 's',
 '▁d',
 "'",
 'Organ',
 'isation',
 '.',
 ')',
 ',',
 '▁the',
 '▁mind',
 '▁map',
 '▁(',
 'or',
 '▁heuristic',
 '▁diagram',
 ')',
 '▁is',
 '▁a',
 '▁graphic',
 '▁representation',
 '▁technique',
 '▁that',
 '▁follows',
 '▁the',
 '▁natural',
 '▁functioning',


In [50]:
word_label_mapping

{'your': tensor([0, 1, 1, 1, 0, 0, 1])}

In [54]:
filtered_words

['your', 'original', 'sentence', 'here']

In [52]:
word_label_mapping = dict(zip(filtered_words, predicted_labels))

In [53]:
word_label_mapping

{'your': tensor([0, 1, 1, 1, 0, 0, 1])}

In [49]:
filtered_words

['your', 'original', 'sentence', 'here']

# Latest Pipeline

In [48]:
# Create a dictionary to map each word to its predicted label
word_label_mapping = dict(zip(filtered_words, predicted_labels))

# Reconstruct the original string without special tokens
reconstructed_string = tokenizer.convert_tokens_to_string(filtered_words)

# Align predicted labels with the reconstructed string
aligned_labels = [word_label_mapping[word] for word in filtered_words]

print("Original String:", original_string)
print("Reconstructed String:", reconstructed_string)
print("Aligned Labels:", aligned_labels)

KeyError: 'original'